<a href="https://colab.research.google.com/github/alfredofosu/python.projects/blob/main/_01Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we utilize historical meteorological and hydrometric data obtained from the Meteorological Services Canada (MSC), which is a division of the Environment and Climate Change Canada (ECCC). The MSC collects and maintains meteorological data from various weather stations across Canada, recording information such as temperature, precipitation, humidity, wind speed, and wind direction on an hourly, daily, and monthly basis. Hydrometric parameters such as waterflow rate and stages are also recorded on the same time scale. These data are publicly accessible and provide valuable insights into past weather conditions.

To access the data, we leverage the MSC GeoMet platform, which offers web services and application programming interfaces (API) for retrieving data from their databases. The API can be accessed using the base URL https://api.weather.gc.ca/ and provides access to different collections or folders containing queryable data.

By utilizing the MSC API, we can programmatically retrieve the desired climate data, enabling us to analyze and explore historical data patterns, understand trends, and extract valuable information for various applications. The availability of this data through an API facilitates seamless integration into data analysis workflows, making it easier for researchers, scientists, and other users to access and utilize the wealth of climate information provided by the MSC.

In [ ]:
#@title Import libraries
import requests
import pandas as pd
from tqdm import tqdm

# dt_now = pd.Timestamp.today(tz='US/Eastern')

# change into workingspace
%cd /content/drive/MyDrive/Colab_Notebooks/master_thesis

/content/drive/MyDrive/Colab_Notebooks/master_thesis


In [ ]:
wq_data_trca = {"2015_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/df177a2f-b431-4d65-bb88-d5a616088808/download/2015-wq-data-from-database-no-project-sites1.csv",
 "2016_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/df177a2f-b431-4d65-bb88-d5a616088808/download/2015-wq-data-from-database-no-project-sites1.csv",
 "2017_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/2b4dd76e-3fe1-4f35-9013-0e5c8aa2f3ff/download/trca-2017-wq-data-rwmp-sites.csv",
 "2018_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/01ea1697-ed6c-429d-bb98-5b01cccfaac3/download/trca-2018-wq-data-rwmp-sites.csv",
 "2019_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/0e387fec-60e1-4324-ba45-25dcd3615cdd/download/trca-2019-wq-data-rwmp-sites-1.csv",
 "2020_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/01ea1697-ed6c-429d-bb98-5b01cccfaac3/download/trca-2020-wq-data-rwmp-sites.csv",
 "2021_TRCA":"https://data.trca.ca/dataset/48e50166-f641-4084-bd40-ecc480cd8d1b/resource/6f56985a-09af-440b-a2ee-eeb432d7e255/download/2021-wq-data-1.csv",
}

In [ ]:
wq_data_trca_list = []
for year, data in wq_data_trca.items():
  df = pd.read_csv(data, encoding='windows-1252')
  wq_data_trca_list.append(df)

In [ ]:
df= pd.concat(wq_data_trca_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204732 entries, 0 to 30431
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   StationID                204732 non-null  object 
 1   ShortName                204732 non-null  object 
 2   TRCA_StationID           110183 non-null  object 
 3   StationMapName           172847 non-null  object 
 4   Watershed                204732 non-null  object 
 5   UTMNorthing              204732 non-null  float64
 6   UTMEasting               204732 non-null  float64
 7   UTMDatum                 183102 non-null  object 
 8   Result_Date              204732 non-null  object 
 9   Year                     204732 non-null  int64  
 10  Month                    204732 non-null  int64  
 11  Day                      204732 non-null  int64  
 12  ParameterDefinitionName  204732 non-null  object 
 13  ParameterGroup           204732 non-null  object 
 14  Final

In [ ]:
with open(f"./data/monitoring_stations_data/water_quality/raw_data/wq_trca_raw_long_form.csv",'w') as file_csv:
      df.to_csv(file_csv, index=False)

In [ ]:
df_new = df[['StationID','Watershed','ParameterDefinitionName','Result_Date','FinalValue', 'FinalUnits', 'Result_Qualifier', 'DuplicateCode',]]

# Meteorological Data Extraction

In [ ]:
#@title Extract daily climate observations from MSC and WSC
# Define a function to extract daily meterological and hydrometric observations
def get_observations(station_ids: list,
                     datetime: str,
                     limit: str,
                     domain: str):
  # Create empty list to store all data retrieved from API
  all_data = []

  # Loop through the climate identifiers if more than one
  for stationId in station_ids:
    global url, headers, params

    if domain == 'climate':
        # Set up initial URL and headers for API request
        url = "https://api.weather.gc.ca/collections/climate-daily/items"  # meteorological data
        headers = {"content-type": "application/json"}
        params = {"datetime": datetime,
                  "limit": limit,
                  "sortby": ["PROVINCE_CODE", "CLIMATE_IDENTIFIER", "LOCAL_DATE"],
                  "CLIMATE_IDENTIFIER": stationId}
    elif domain == 'hydrometric':
        # Set up initial URL and headers for API request
        url = "https://api.weather.gc.ca/collections/hydrometric-daily-mean/items"  # hydrometric data
        headers = {"content-type": "application/json"}
        params = {"datetime": datetime,
                  "limit": limit,
                  "PROV_TERR_STATE_LOC": "ON",
                  "sortby": ["PROV_TERR_STATE_LOC", "STATION_NUMBER", "DATE"],
                  "STATION_NUMBER": stationId}

    # Send initial request to API
    response = requests.get(url, headers=headers, params=params)

    # Extract data from response and add to list
    data = response.json()
    all_data.extend(data["features"])

    # Check if there is a "next" URL in the response
    while "next" in [next['rel'] for next in data["links"] if next['rel'] == "next"]:
        # Set URL to the "next" URL in the response
        next_url = [next["href"] for next in data["links"] if "next" in next.values()][0]

        # Send request to "next" URL
        response = requests.get(next_url)

        # Extract data from response and add to list
        data = response.json()
        all_data.extend(data["features"])

  # Print out number of data points retrieved
  print(f"Retrieved {len(all_data)} data points.")

  return all_data


In [ ]:
%%time
daily_climate_data = get_observations(
    station_ids= [
        "6154951", "6154950", "6157012", "6157014", "615S001", "6158718", "6158778", "6158830", "6158550", "6158846",
        "6158567", "615HHDF", "6158779", "6158730", "6158732", "6158417", "6158751", "6158505", "6158412", "6158406",
        "6158780", "6159459", "6158815", "6158420", "61587PG", "6158753", "6158734", "6158255", "6158480", "6154994",
        "61587PP", "6158355", "6158762", "61583FL", "6158474", "6158386", "6158443", "6158398", "6152235", "6158740",
        "6154135", "6155680", "6154131", "6159510", "6157015", "6152953", "615HMAK", "6158409", "6158M1K", "6158422",
        "6158384", "61587PR", "6158765", "6158536", "6158381", "6158575", "6158731", "6158733", "6158578", "6158385",
        "6158408", "6158752", "61584J2"],
    datetime="1964-01-01 00:00:00/2022-12-31 11:59:59", # "1964-01-01 00:00:00/2022-12-31 11:59:59"
    limit="10000",
    domain="climate")

Retrieved 245463 data points.
CPU times: user 4.53 s, sys: 339 ms, total: 4.87 s
Wall time: 1min 50s


In [ ]:
%%time
daily_hydrometric_data = get_observations(
    station_ids=["02HC024", "02HC029", "02HC056", "02HC005", "02HC004"],
    datetime="1964-01-01/2022-12-31",  # "1964-01-01/2022-12-31"
    limit="10000",
    domain="hydrometric")

Retrieved 58777 data points.
CPU times: user 605 ms, sys: 54.1 ms, total: 660 ms
Wall time: 45.8 s


In [ ]:
# Function to convert the dictionary to a Pandas DataFrame
def dict_to_dataframe(data: list) -> pd.DataFrame:
    # Create empty list to store the atmospheric properties
    temp = []

    # Loop through the climate data's properties and append the temp list
    for dic in data:
      temp_dic = dic["properties"]
      temp_dic.update({'Latitude': dic['geometry']['coordinates'][1],
                                  'Longitude': dic['geometry']['coordinates'][0]})
      # 'geometry': {'coordinates': [-79.36150360107422, 43.68585968017578],
      temp.append(temp_dic)

    return pd.DataFrame(temp)

In [ ]:
%%time
daily_climate_df = dict_to_dataframe(daily_climate_data)
# save the extraced files to a designated path/folder
with open(f"./data/monitoring_stations_data/meteorology/daily_climate_data_1964_2021.csv",
          'w') as file_csv:
    daily_climate_df.to_csv(file_csv, index=False)

# save the pandas information to a designated path/folder
with open(f"./data/monitoring_stations_data/meteorology/daily_climate_metadata_1964_2021.txt",
          'w') as file_txt:
    daily_climate_df.info(verbose=True, buf=file_txt)

daily_climate_df.set_index("LOCAL_DATE").sort_index(axis=0, ascending=True).info()

<class 'pandas.core.frame.DataFrame'>
Index: 245463 entries, 1964-01-01 00:00:00 to 2022-12-31 00:00:00
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   STATION_NAME              245463 non-null  object 
 1   CLIMATE_IDENTIFIER        245463 non-null  object 
 2   ID                        245463 non-null  object 
 3   PROVINCE_CODE             245463 non-null  object 
 4   LOCAL_YEAR                245463 non-null  int64  
 5   LOCAL_MONTH               245463 non-null  int64  
 6   LOCAL_DAY                 245463 non-null  int64  
 7   MEAN_TEMPERATURE          126910 non-null  float64
 8   MEAN_TEMPERATURE_FLAG     1403 non-null    object 
 9   MIN_TEMPERATURE           127128 non-null  float64
 10  MIN_TEMPERATURE_FLAG      1384 non-null    object 
 11  MAX_TEMPERATURE           127209 non-null  float64
 12  MAX_TEMPERATURE_FLAG      1195 non-null    object 
 13  TOTAL_PRECIPITATIO

In [ ]:
%%time
daily_hydrometric_df = dict_to_dataframe(daily_hydrometric_data)
# save the extraced files to a designated path/folder
with open(f"./data/monitoring_stations_data/hydrometric/daily_hydrometric_data_1964_2021.csv",
          'w') as file_csv:
    daily_hydrometric_df.to_csv(file_csv, index=False)

# save the pandas information to a designated path/folder
with open(f"./data/monitoring_stations_data/hydrometric/daily_hydrometric_metadata_1964_2021.txt",
          'w') as file_txt:
    daily_hydrometric_df.info(verbose=True, buf=file_txt)

daily_hydrometric_df.set_index("DATE").sort_index(axis=0, ascending=True).info()

<class 'pandas.core.frame.DataFrame'>
Index: 58777 entries, 1964-01-01 to 2021-12-31
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDENTIFIER           58777 non-null  object 
 1   STATION_NAME         58777 non-null  object 
 2   STATION_NUMBER       58777 non-null  object 
 3   PROV_TERR_STATE_LOC  58777 non-null  object 
 4   LEVEL                19848 non-null  float64
 5   DISCHARGE            55347 non-null  float64
 6   DISCHARGE_SYMBOL_EN  6082 non-null   object 
 7   DISCHARGE_SYMBOL_FR  6082 non-null   object 
 8   LEVEL_SYMBOL_EN      257 non-null    object 
 9   LEVEL_SYMBOL_FR      257 non-null    object 
 10  Latitude             58777 non-null  float64
 11  Longitude            58777 non-null  float64
dtypes: float64(4), object(8)
memory usage: 5.8+ MB
CPU times: user 1.14 s, sys: 10.5 ms, total: 1.15 s
Wall time: 1.21 s


#Hydrometric Data Extraction

In [ ]:
#@title Extract hisotrical hydrometric data from the TRCA
stage_base_url = 'https://data.trca.ca/dataset/7af8f431-791c-4ffc-ad6c-f2c006207e35/resource'

stage = {'HY017': '/95684835-21c9-4297-b156-e2bfb6fddc7a/download/stage-finalhy017.csv',
         'HY018': '/6644fca1-4a2a-45bb-8a12-c69a6c28804c/download/stage-finalhy018.csv',
         'HY019': '/9b63daa8-9180-4e38-b6bf-b6b3417e1e79/download/stage-finalhy019.csv',
         'HY022': '/3e562c44-1e19-4935-be8a-746955fe1625/download/stage-finalhy068.csv',
         'HY027': '/2d9232ab-ee9b-443b-8ca3-681af1286b40/download/stage-finalhy062.csv',
         'HY062': '/2d9232ab-ee9b-443b-8ca3-681af1286b40/download/stage-finalhy062.csv',
         'HY068': '/3e562c44-1e19-4935-be8a-746955fe1625/download/stage-finalhy068.csv',
         'HY079': '/991a6462-e16e-4479-8878-e23121af6453/download/stage-finalhy079.csv',
         'HY080': '/b1f40f56-cb88-401c-9481-996c619cf9fe/download/stage-finalhy080.csv',
         'HY092': '/3daf087a-0d6c-43a1-b5da-42a4692aeee7/download/stage-finalhy092.csv',
         'HY093': '/019a2330-32ae-4f75-94ed-41f299a7bfcb/download/stage-finalhy093.csv',
         'HY100': '/3a244fc6-6403-447c-9fe3-aab77ed21560/download/stage-finalhy100.csv',
         'HY112': '/6beb6e45-278a-4187-a528-287162191f0a/download/stage-finalhy112.csv',
         'HY123': '/91b38e64-9acc-45a0-967a-012898fb9640/download/stage-finalhy123.csv',
         }

discharge_base_url = 'https://data.trca.ca/dataset/f7230f5d-3fb1-44c9-9567-53e62a442476/resource'

discharge = {'HY017': '/c518811a-f8cc-47c5-9e4d-afd729a2c8fa/download/discharge-finalhy017.csv',
             'HY018': '/a37bddd5-ea58-4142-8978-670f52e9d742/download/discharge-finalhy018.csv',
             'HY062': '/5b306ffc-18c9-4af3-af2a-8256bd2a3049/download/discharge-finalhy062.csv',
             'HY092': '/c5a18273-0948-46d6-af08-252569ffebd4/download/discharge-finalhy092.csv',
             'HY093': '/9fd2dae9-0b98-47d5-b1d5-3042e0ae8d40/download/discharge-finalhy093.csv',
             'HY100': '/9e257afc-66f1-4cf0-9b03-e4fb1b5cb3cc/download/discharge-finalhy100.csv',
             'HY112': '/06afe410-4f2f-4404-ba79-970edd9d9eb2/download/discharge-finalhy112.csv',
             }

precipitation_base_url = 'https://data.trca.ca/dataset/fa58913b-a057-4785-b6c2-28c6c35d535d/resource'

precipitation = {'HY008': '/0c829fc4-7cde-4598-b877-680431f734e9/download/precip-total-finalhy008.csv',
                 'HY016': '/6fd94945-8705-4033-9541-f6640a5bf818/download/precip-total-finalhy016.csv',
                 'HY021': '/b400096a-16ef-40e7-97a0-ec58a81f9746/download/precip-total-finalhy021.csv',
                 'HY027': '/879358aa-f1a0-4064-8f82-2434ee4d3d8a/download/precip-total-finalhy027.csv',
                 'HY069': '/4b435994-b60b-454c-a87f-094adb7b8fb4/download/precip-total-finalhy069.csv',
                 'HY085': '/9f545506-f270-4dc0-bf0d-a8b515ee9890/download/precip-total-finalhy085.csv',
                 'HY094': '/cb4d20ea-f152-4272-ae09-6b0fa9ff0a60/download/precip-total-finalhy094.csv',
                 'HY036': '/b63b31e0-7685-46fe-a8b7-a9422d602552/download/precip-total-finalhy036.csv',
                 'HY038': '/321d0f62-360e-4d98-99c7-a59ed4e02e45/download/precip-total-finalhy038.csv',
                 'HY070': '/1c0ab50a-c22a-4ee2-912c-ee4997006229/download/precip-total-finalhy070.csv',
                 'HY039': '/627f4dcd-a895-4a73-b41f-ece417f3bf07/download/precip-total-finalhy039.csv',
                 'HY055': '/2d1707f2-aabe-4934-8a7a-2096acb87bda/download/precip-total-finalhy055.csv',
                 'HY064': '/fe3353dc-5758-4369-be2b-84071b1f1cce/download/precip-total-finalhy064.csv',
                 'HY087': '/fada2642-06a9-4835-af3c-34bc8fcc8564/download/precip-total-finalhy087.csv',
                 'HY003': '/b264812c-adc0-4622-b930-b3ab0bf024e6/download/precip-total-finalhy003.csv',
                 }

In [ ]:
def download_hydrometric_data_trca(dic: dict,base_url: str, param:str , agency: str) -> pd.DataFrame:
  temp_list = []
  for station_id, url in tqdm(dic.items()):

    df = pd.read_csv(base_url+url, low_memory=False)

    # search for the row index that contains the "timestamp"
    idx = df.index[df["LocationIdentifier"].str.contains('timestamp')][0]

    # create a new dataframe starting from the row that containst the word "timestamp"
    df_new = pd.DataFrame(df.iloc[idx + 1:].values, columns=["datetime", "result", "unit",])

    df_new["datetime"] = pd.to_datetime(df_new["datetime"])
    df_new["result"] = df_new["result"].astype("float32")

    df_new = df_new.set_index("datetime").resample("D").agg(
        num_reading_per_day=("result", "count"),
        avg_daily=("result", "mean")
    )

    # add the unit and station name to the data
    df_new['unit'] = str(df.iloc[0,2])
    df_new['station_id']= str(df.iloc[0,0])

    # sort and reset index
    df_new.sort_index(inplace=True)
    df_new.reset_index(inplace=True)
    df_new = df_new.rename(columns={"avg_daily": f"avg_daily_{param}"})

    temp_list.append(df_new)

    # save the extraced files to a designated path/folder
    with open(f"./data/monitoring_stations_data/hydrometric/{agency}/{station_id}_daily_{param}.csv",'w') as file_csv:
      df_new.to_csv(file_csv, index=False)

  return pd.concat(temp_list)

In [ ]:
%%time

stage_data_trca = download_hydrometric_data_trca(stage, stage_base_url, "stage", "trca")
stage_data_trca

100%|██████████| 14/14 [00:57<00:00,  4.14s/it]

CPU times: user 40.4 s, sys: 1.05 s, total: 41.5 s
Wall time: 59.1 s


,datetime,num_reading_per_day,avg_daily_stage,unit,station_id
0,2012-12-05 00:00:00-05:00,60,180.807556,m,HY017
1,2012-12-06 00:00:00-05:00,96,180.758743,m,HY017
2,2012-12-07 00:00:00-05:00,96,180.748764,m,HY017
3,2012-12-08 00:00:00-05:00,96,180.795242,m,HY017
4,2012-12-09 00:00:00-05:00,96,180.765305,m,HY017
...,...,...,...,...,...
227,2021-02-27 00:00:00-05:00,96,98.426384,m,HY123
228,2021-02-28 00:00:00-05:00,96,98.421265,m,HY123
229,2021-03-01 00:00:00-05:00,96,98.419228,m,HY123
230,2021-03-02 00:00:00-05:00,96,98.410362,m,HY123


In [ ]:
discharge_data_trca = download_hydrometric_data_trca(discharge, discharge_base_url, "discharge", "trca")
discharge_data_trca

100%|██████████| 7/7 [00:24<00:00,  3.50s/it]


,datetime,num_reading_per_day,avg_daily_discharge,unit,station_id
0,2012-12-05 00:00:00-05:00,60,0.552094,m^3/s,HY017
1,2012-12-06 00:00:00-05:00,96,0.310023,m^3/s,HY017
2,2012-12-07 00:00:00-05:00,96,0.266037,m^3/s,HY017
3,2012-12-08 00:00:00-05:00,96,0.494216,m^3/s,HY017
4,2012-12-09 00:00:00-05:00,96,0.340379,m^3/s,HY017
...,...,...,...,...,...
256,2021-01-03 00:00:00-05:00,96,0.459833,m^3/s,HY112
257,2021-01-04 00:00:00-05:00,96,0.393911,m^3/s,HY112
258,2021-01-05 00:00:00-05:00,96,0.324771,m^3/s,HY112
259,2021-01-06 00:00:00-05:00,96,0.283813,m^3/s,HY112


In [ ]:
precipitation_data_trca = download_hydrometric_data_trca(precipitation, precipitation_base_url, "precip", "trca")
precipitation_data_trca

100%|██████████| 15/15 [02:50<00:00, 11.36s/it]


,datetime,num_reading_per_day,avg_daily_precip,unit,station_id
0,2009-06-23 00:00:00-05:00,120,0.000000,mm,HY008
1,2009-06-24 00:00:00-05:00,288,0.000000,mm,HY008
2,2009-06-25 00:00:00-05:00,288,0.028472,mm,HY008
3,2009-06-26 00:00:00-05:00,288,0.000000,mm,HY008
4,2009-06-27 00:00:00-05:00,288,0.000000,mm,HY008
...,...,...,...,...,...
5670,2020-11-06 00:00:00-05:00,288,0.000000,mm,HY003
5671,2020-11-07 00:00:00-05:00,288,0.000000,mm,HY003
5672,2020-11-08 00:00:00-05:00,288,0.000000,mm,HY003
5673,2020-11-09 00:00:00-05:00,288,0.000000,mm,HY003


# Water Temperature Data

In [ ]:
water_temperature_base_url = 'https://data.trca.ca/dataset/1785e637-6a45-4de0-8375-1116ffd05b2d/resource'
water_temperature = {
    '2002': '/79e236c6-cf96-444f-9b3a-40a744962cc8/download/rwmp_watertemperature_data_2002.csv',
    '2003': '/9fc70d30-7782-4a8a-9113-7a87adaf9145/download/rwmp_watertemperature_data_2003.csv',
    '2004': '/5e833ba9-6aa6-4bcc-b7d4-76eb7ee5db90/download/rwmp_watertemperature_data_2004.csv',
    '2005': '/9c630284-fb20-4f75-b8f4-350380a5769d/download/rwmp_watertemperature_data_2005.csv',
    '2006': '/89a8fe69-39e4-46f4-ad41-601fb07a411e/download/rwmp_watertemperature_data_2006.csv',
    '2007': '/8398dca5-a98d-4485-b984-557eb5d87a82/download/rwmp_watertemperature_data_2007.csv',
    '2008': '/059237b1-5cab-471e-81eb-6e13ea8c40da/download/rwmp_watertemperature_data_2008.csv',
    '2009': '/e05c0b64-edb8-413e-b544-2887f8e49d9c/download/rwmp_watertemperature_data_2009.csv',
    '2010': '/05bb2242-7462-4a30-ab3b-4d63723b4406/download/rwmp_watertemperature_data_2010.csv',
    '2011': '/9dfbf872-af3a-4124-81c2-97f35b37a14b/download/rwmp_watertemperature_data_2011.csv',
    '2012': '/f4b624b8-b0ce-419d-816a-ec349cfe4a52/download/rwmp_watertemperature_data_2012.csv',
    '2013_a': '/12fd7f01-0d9f-42e9-8b1d-40818a792f6b/download/rwmp_watertemperature_data_2013.csv',
    '2013_b': '/0485ddd2-d6dd-4212-89d5-6b4bb82887df/download/rwmp_watertemperature_data_2013.csv',
    '2014': '/1149c506-2bfe-47e4-9227-eb9957937ebe/download/rwmp_watertemperature_data_2014.csv',
    '2015': '/4f5032ed-2123-4ca4-bec2-f08c614141f5/download/rwmp_watertemperature_data_2015.csv',
    '2016': '/7890b473-527d-4bc0-8acf-292baced46c3/download/rwmp_watertemperature_data_2016.csv',
    '2017_a': '/8061cce0-27e7-483d-ae5a-2aabfc2c661b/download/rwmp_watertemperature_data_2017.csv',
    '2017_b': '/4d53f8b1-e48c-4648-9a45-39faac9e7875/download/rwmp_watertemperature_data_2017.csv',
    '2018_a': '/b885c4a5-1b1d-4d55-bd9c-2bf985a634bd/download/rwmp_watertemperature_data_2018_a.csv',
    '2018_b': '/b3e292d3-b619-4635-b1b0-a4d7cfef2261/download/rwmp_watertemperature_data_2018_b.csv',
    '2018_c': '/5428e462-e647-4d94-971c-4dd04186d556/download/rwmp_watertemperature_data_2018_c.csv',
    '2019': '/61277f0a-596d-4826-8f96-70909673e56d/download/rwmp_watertemperature_data_2019.csv',
    '2020': '/5cde5b5e-5095-4d4b-87c3-4705fce4f852/download/rwmp_watertemperature_data_2020.csv',
}

# def get_water_temperature(data_urls:dict, base_url: str):
#   temp=[]
#   for year, url in tqdm(data_urls.items()):
#     # Some datasets have no column names, so this would need to be extracred using location indices.
#     if year == '2018_b' or year =='2018_c':

#       df = pd.read_csv(base_url+url, low_memory=False, on_bad_lines='skip', header=None)

#       df.iloc[:,17] = pd.to_datetime(df.iloc[:, 17])

#       df.loc[:,25] = df.iloc[:,17].dt.date

#       df = df.iloc[:, [5, 7, 11 , 12 , 25 , 21]]

#       df = df.loc[df[7] == "Don"]

#       df = df.groupby([5, 7, 11 , 12 , 25]).mean()

#       df.reset_index(inplace=True)

#       df = df.rename(columns={5:"Operator_SiteID", 7: "Watershed", 11: "Latitude", 12: "Longitude", 25: "Date", 21: "Obs_Value"})
#     else:
#       df = pd.read_csv(base_url+url, low_memory=False, on_bad_lines='skip',)

#       df["TimeTag"] = pd.to_datetime(df["TimeTag"])

#       df.loc[:,"Date"] = df["TimeTag"].dt.date

#       df = df[["Operator_SiteID", "Watershed", "Latitude", "Longitude", "Date", "Obs_Value"]]

#       df = df.loc[df["Watershed"] == "Don"]

#       df = df.groupby(["Operator_SiteID", "Watershed", "Latitude", "Longitude", "Date",]).mean()

#       df.reset_index(inplace=True)

#     temp.append(df)
#     # save the pandas information to a designated path/folder
#     with open(f"./data/monitoring_stations_data/hydrometric/{year}_water_temperature_data.csv",'w') as file_csv:
#       df.to_csv(file_csv, index=False)
#   df_all = pd.concat(temp)
#   return df_all.reset_index(drop=True)

In [ ]:
#@title Using Multiprocessing to download data
import concurrent.futures

def get_water_temperature(data_urls:dict, base_url: str):
  temp = []
  def process_data(year, url):
    # Some datasets have no column names, so this would need to be extraced using location indices.
    if year == '2018_b' or year =='2018_c':
      df = pd.read_csv(base_url+url, low_memory=False, on_bad_lines='skip', header=None)
      df.iloc[:, 17] = pd.to_datetime(df.iloc[:, 17])
      df.loc[:, 25] = df.iloc[:, 17].dt.date
      df = df.iloc[:, [5, 7, 11, 12, 25, 21]]
      df = df.loc[df[7] == "Don"]
      df = df.groupby([5, 7, 11, 12, 25]).mean()
      df.reset_index(inplace=True)
      df = df.rename(columns={5: "Operator_SiteID", 7: "Watershed", 11: "Latitude", 12: "Longitude", 25: "Date", 21: "Obs_Value"})
    else:
      df = pd.read_csv(base_url+url, low_memory=False, on_bad_lines='skip')
      df["TimeTag"] = pd.to_datetime(df["TimeTag"])
      df.loc[:,"Date"] = df["TimeTag"].dt.date
      df = df[["Operator_SiteID", "Watershed", "Latitude", "Longitude", "Date", "Obs_Value"]]
      df = df.loc[df["Watershed"] == "Don"]
      df = df.groupby(["Operator_SiteID", "Watershed", "Latitude", "Longitude", "Date"]).mean()
      df.reset_index(inplace=True)

    # save the pandas information to a designated path/folder
    with open(f"./data/monitoring_stations_data/hydrometric/{year}_water_temperature_data.csv", 'w') as file_csv:
        df.to_csv(file_csv, index=False)

    return df

  with concurrent.futures.ThreadPoolExecutor() as executor:
      with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_data, year, url) for year, url in data_urls.items()]

        for future in tqdm(concurrent.futures.as_completed(futures)):
            temp.append(future.result())

  return pd.concat(temp)

In [ ]:
wt_df = get_water_temperature(water_temperature, water_temperature_base_url)

23it [02:27,  6.43s/it]


In [ ]:
wt_df

,Operator_SiteID,Watershed,Latitude,Longitude,Date,Obs_Value
0,DN003WM,Don,43.7093,-79.2810,2003-07-18,17.976937
1,DN003WM,Don,43.7093,-79.2810,2003-07-19,16.814021
2,DN003WM,Don,43.7093,-79.2810,2003-07-20,16.231792
3,DN003WM,Don,43.7093,-79.2810,2003-07-21,16.688688
4,DN003WM,Don,43.7093,-79.2810,2003-07-22,18.171479
...,...,...,...,...,...,...
557,DN023WM,Don,43.8509,-79.4499,2019-12-27,3.356052
558,DN023WM,Don,43.8509,-79.4499,2019-12-28,3.217469
559,DN023WM,Don,43.8509,-79.4499,2019-12-29,2.899365
560,DN023WM,Don,43.8509,-79.4499,2019-12-30,3.570208


In [ ]:
wt_df
with open(f"./data/monitoring_stations_data/hydrometric/water_temperature_data_trca.csv",'w') as file_csv:
  wt_df.to_csv(file_csv, index=False)

In [ ]:
wt_df.loc[wt_df.iloc[:,0].str.contains('DN')]

,Operator_SiteID,Latitude,Longitude,Date,Obs_Value
0,DN001WM,43.6792,-79.3642,2002-05-24,16.696500
1,DN001WM,43.6792,-79.3642,2002-05-25,14.167563
2,DN001WM,43.6792,-79.3642,2002-05-26,14.218562
3,DN001WM,43.6792,-79.3642,2002-05-27,15.071083
4,DN001WM,43.6792,-79.3642,2002-05-28,16.358917
...,...,...,...,...,...
219173,DN023WM,43.8509,-79.4499,2020-08-02,21.374229
219174,DN023WM,43.8509,-79.4499,2020-08-03,20.641281
219175,DN023WM,43.8509,-79.4499,2020-08-04,20.433104
219176,DN023WM,43.8509,-79.4499,2020-08-05,19.634010
